# Goal

- [ ] review `git` activity from the past `x` days/weeks/months (my main goal is weekly but I want the program to be flexible to use for quarterly and annual review) ('last week done' and 'repo activity' sections)
- [ ] review Apple notes changes?? this is my primary notetaking program but I've tried to access the data before and wasn't able to. It would be great to track changes in pinned notes and flag unchecked tasks without having to enter them ('last week done', 'carryover from last week', capture free text notes that might be worth briefing others about)
- [ ] process a preview weekly for incomplete tasks and prompt user to mark completion as Yes/No/Progress for each task ('last week done' and 'carryover from last week' sections)

I think I already have a way to
- [ ] import this week's meetings from calendar (filter out holiday calendar, events outside of 9am-5pm PT)

# `git` activity review

- probably best to use the [`gitpython`](https://gitpython.readthedocs.io/en/stable/tutorial.html#tutorial-label) module

## setup

In [1]:
# dependencies
from os import listdir, path
from pathlib import Path
from datetime import date, datetime
from dateutil.relativedelta import *
import git
import re

In [2]:
# support methods
def findrepos(gitdir):
    gitdir = path.expanduser(gitdir)
    info = {}
    for dirname in listdir(gitdir):
        dirpath = f"{gitdir}/{dirname}"
        if Path(f"{dirpath}/.git").exists(): info[dirname] = {'path': dirpath}
    assert info
    return info


def checkrepos(info):
    newinfo = info
    for reponame, repoinfo in info.items():
        repo = git.Repo(repoinfo['path'])
        newinfo[reponame]['dirty'] = repo.is_dirty()
        newinfo[reponame]['untracked'] = [
            f for f in repo.untracked_files if 'checkpoint' not in f]
    assert newinfo
    return newinfo


def recentcommits(info, sdate, edate, author=None):
    """Authored datetime is preserved on rebase, and
    we want to include commits from this week that might have been rebasing an earlier commit."""
    newinfo = info
    for reponame, repoinfo in info.items():
        repo = git.Repo(repoinfo['path'])
        if author: newinfo[reponame]['n_other_recent'] = 0
        commits = []
        nother = 0
        for commit in repo.iter_commits():
            tzaware = commit.committed_date + commit.committer_tz_offset
            committed = datetime.fromtimestamp(tzaware)
            if (committed < edate) & (committed >= sdate):
                if author:
                    if author.lower() in commit.author.name.lower(): commits.append(commit)
                    else: newinfo[reponame]['n_other_recent'] += 1
                else: commits.append(commit)
        newinfo[reponame]['recent'] = commits
        if newinfo[reponame]['n_other_recent'] == 0: newinfo[reponame].pop('n_other_recent')
    assert newinfo
    return newinfo


def formatmessage(msg):
    """I want the first work of the commit message to be title-cased, but not the rest of the message."""
    chunks = msg.strip().split()
    titled = chunks[0].title() + ' ' + ' '.join(chunks[1:])
    if titled[-1] != ".": titled += "."
    return titled


def summarize(reponame, commits):
    if not any(commits): return ""
    summary = ""
    for commit in commits:
        commitdt = datetime.fromtimestamp(commit.committed_date + commit.committer_tz_offset).strftime("%a %d %b")
        nchanges = commit.stats.total
        overview = f"- [{commit.hexsha[:8]}]:\t{formatmessage(msg=commit.message)}"
        overview += f"\n\t\t// Involves {nchanges['files']} file(s), {nchanges['lines']} lines // "
        overview += f"Committed: {commitdt}\n"
        summary += overview
    summary += "\n"
    return summary


def summarizerecent(repos):
    fullsummary = ""
    for reponame, repoinfo in repos.items():
        if not ((any(repoinfo['recent'])) | ('n_other_recent' in repoinfo.keys())): continue
        summary = f"Commits in `{reponame}`:\n"
        if any(repoinfo['recent']):
            recent = summarize(reponame=reponame, commits=repoinfo['recent'])
            summary += recent
        if 'n_other_recent' in repoinfo.keys():
            if repoinfo['n_other_recent'] > 0:
                summary += f"- {repoinfo['n_other_recent']} commits by other users.\n"
        fullsummary = fullsummary + "\n" + summary
    return fullsummary

In [3]:
# main
base = findrepos(gitdir="~/git")
base = checkrepos(info=base)
today = datetime.now()
aweekago = today - relativedelta(days=+7)
repos = recentcommits(info=base, sdate=aweekago, edate=today, author="bailey")
summary = summarizerecent(repos)

## preview

In [4]:
reponame = 'Chi-MP-data-story'
repo = git.Repo(repos[reponame]['path'])
recent = repos[reponame]['recent']
commit = recent[0]

In [5]:
commit

<git.Commit "ed746774c471ae3c64de57fb29997f286643f7a6">

In [6]:
repo.commit().author.name

'Bailey'

In [7]:
commit.message

'pulled one more small section and re-ran notebook\n'

In [8]:
print(summary)


Commits in `Chi-MP-data-story`:
- [ed746774]:	Pulled one more small section and re-ran notebook.
		// Involves 1 file(s), 383 lines // Committed: Thu 09 Jan
- [bdbfc9b9]:	Finished incorporating feedback from TS and MEP; need to work on 2 titles otherwise ready to pass to II.
		// Involves 2 file(s), 1507 lines // Committed: Thu 09 Jan


Commits in `US-Ventura-RJA`:
- [0a795b58]:	Added other sequence fields that can be used to identify a particular version of a case against a defendant.
		// Involves 1 file(s), 7 lines // Committed: Mon 06 Jan


